In [1]:
import sys
from pathlib import Path

IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules
if IS_KAGGLE:
    repo_path = Path("../input/crypto-prediction")
elif IS_COLAB:
    from google.colab import drive

    drive.mount("/content/gdrive")
    repo_path = Path("/content/gdrive/MyDrive/crypto-prediction")
else:
    repo_path = Path("/home/matias/crypto-prediction")
sys.path.append(str(repo_path))

import asyncio
import os
import pickle
import re
import sys
import time
from dataclasses import dataclass
from datetime import datetime, timedelta
from importlib import reload
from pathlib import Path
from pprint import pprint
from typing import Callable, List, Optional, Union

import ccxt
import get_data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytz
import talib
import tensorflow as tf
import wandb
import yfinance as yf
from binance import AsyncClient, BinanceSocketManager
from binance.client import Client
from binance.enums import *
from binance.exceptions import BinanceAPIException
from binance.helpers import round_step_size
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from tensorflow.keras import layers
from tensorflow.keras.activations import sigmoid, tanh
from tensorflow.keras.models import Model
from tools import inspect_code, plotting, training, wandb_api
from tools.training import Datapoint
from tqdm import tqdm
from wandb.keras import WandbCallback
from portfolio import portfolio

log_wandb = True
tf.config.list_physical_devices("CPU")[0].device_type
starting_date = datetime.now()


2022-04-11 22:46:33.632919: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-11 22:46:33.632975: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-11 22:46:36.026158: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-11 22:46:36.026199: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-11 22:46:36.026215: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Matias): /proc/driver/nvidia/version does not exist


In [2]:
wandb_api.login()
api = wandb.Api()
run_name = "1fullt5y"
run = api.run(f"matiasetcheverry/crypto-prediction/{run_name}")
config = run.config

root_path = Path("tmp")
datamodule_script = run.file("datamodule_script.txt")
datamodule_script = datamodule_script.download(root=root_path / run.name, replace=True)
datamodule_script.seek(0)
datamodule_name = re.findall(r"(?<=class ).[a-zA-Z0-9_.-]*", datamodule_script.read())[
    0
]
datamodule_script.seek(0)

with open(datamodule_script.name, "r") as file:
    exec(file.read())

model = run.file("rf.pkl")
model = model.download(root=root_path / run.name, replace=True)
with open(model.name, "rb") as file:
    rf = pickle.load(file)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: matiasetcheverry (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/matias/.netrc


In [3]:
class ClassificationPortfolio(portfolio.Portfolio):
    def compute_amount_money(
        self, money: float, directions: np.ndarray, probabilities: np.ndarray
    ):
        # preallocated_money = [
        #     float(self.client.get_asset_balance(asset=asset.ticker)["free"])
        #     - asset.initial_amount
        #     for asset in self.assets.values()
        # ]
        allocated_money = np.zeros_like(probabilities)
        # mask = directions * (np.array(preallocated_money) == 0)
        mask = directions
        allocated_money[mask] = (
            probabilities[mask] * money / np.sum(probabilities[mask])
        )
        return allocated_money

    def compute_probabilities(self) -> np.ndarray:
        probabilities = []
        for asset in self.assets.values():
            try:
                probabilities.append(asset.predict_proba_last_from(rf))
            except ValueError as e:
                print(f"Problem predicting on {asset.ticker}")
                print(f"\t{e}")
                probabilities.append(0)
        return np.array(probabilities)


    def trade(self, money: int):
        self.cancel_orders()
        probabilities = self.compute_probabilities()
        directions = probabilities > 0.5
        allocated_money = self.compute_amount_money(
            money,
            directions,
            probabilities,
        )
        order_ids = []
        for index, asset in enumerate(self.assets.values()):
            probability = probabilities[index]
            direction = directions[index]
            money = allocated_money[index]
            orders = self.swap_single_ticker(direction, money, asset)
            if orders is not None:
                for order in orders:
                    print("New order")
                    print(f"\tTICKER: {order['symbol']}")
                    print(f"\tSide: {order['side']}")
                    print(f"\tProba going up: {probability}")
                    print(f"\tAmount: {order['origQty']}")
                    print(f"\tPrice: {order['price']}")
                    print(
                        f"\tMoney allocated: {float(order['origQty'])*float(order['price'])}"
                    )
                    print(f"\tOrder id: {order['orderId']}")
                    print(f"\tTransaction time: {order['transactTime']}")
                    order_ids.append(
                        {"symbol": order["symbol"], "orderId": order["orderId"]}
                    )
        return order_ids

# reload(portfolio)
# key = "aqyuK7BLhzYiU2SCeALa7wayggR9769RtVenBI57683r7N2YC6bT3KUBwSw9zsGk"
# secret = "XqhAXqp6J7OsgRX40T7NH2iHjFpSegtImWcOAwEwuNWcdQKo4GMrzpGu9tUxQe97"
# client = Client(key, secret, testnet=True)
api_key = os.environ.get("TESTNET_API")
api_secret = os.environ.get("TESTNET_SECRET")
client = Client(api_key, api_secret, testnet=True)
pf = ClassificationPortfolio(client, config, repo_path / "live_trading", ["BNB", "BTC", "ETH", "LTC", "TRX", "XRP"])
# pf.trade(float(pf.client.get_asset_balance(asset="BUSD")["free"]))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [9]:
async def kline_listener(aclient):
    bm = BinanceSocketManager(aclient)
    input_coroutines = [
        asset.kline_listener(bm, interval="5m") for asset in list(pf.assets.values())
    ]
    await asyncio.gather(*input_coroutines, return_exceptions=True)
    print("Finally gathered at:", datetime.now())
    # assert all(
    #     list(dm.datapoints.values())[0].df.shape == dp.df.shape
    #     for dp in dm.datapoints.values()
    # )
    print("Initially traded at:", datetime.now())
    pf.trade(float(pf.client.get_asset_balance(asset="BUSD")["free"]))
    print("Finally traded at:", datetime.now())

    for asset in pf.assets.values():
        asset.update_trades(client, starting_date=starting_date, limit=20)
        asset.update_orders(client, starting_date=starting_date, limit=20)
        asset.save_history(repo_path / "live_trading" / asset.ticker)
    pf.save_balances(repo_path / "live_trading" / "balances.csv")


aclient = await AsyncClient.create()
while True:
    await kline_listener(aclient)
    print("------------------------------------------")


Finally gathered at: 2022-04-12 00:00:08.574788
Initially traded at: 2022-04-12 00:00:08.575027
New order
	TICKER: BNBBUSD
	Side: SELL
	Proba going up: 0.44563736888217437
	Amount: 46.45000000
	Price: 392.90000000
	Money allocated: 18250.205
	Order id: 404905
	Transaction time: 1649714418715
New order
	TICKER: BTCBUSD
	Side: SELL
	Proba going up: 0.4711701501258713
	Amount: 1.58002400
	Price: 39398.86000000
	Money allocated: 62251.14437264
	Order id: 1805150
	Transaction time: 1649714422060
New order
	TICKER: LTCBUSD
	Side: SELL
	Proba going up: 0.4462977227195228
	Amount: 269.99137000
	Price: 102.70000000
	Money allocated: 27728.113699
	Order id: 53944
	Transaction time: 1649714424029
New order
	TICKER: XRPBUSD
	Side: SELL
	Proba going up: 0.4357070865005683
	Amount: 45505.60000000
	Price: 0.68580000
	Money allocated: 31207.740479999997
	Order id: 419152
	Transaction time: 1649714425328
Finally traded at: 2022-04-12 00:00:25.933017
------------------------------------------
Finally ga